In [ ]:
import os
import sys

import cv2
import torch
from matplotlib import pyplot as plt
import segmentation_models_pytorch as smp
import numpy as np
from torch import nn, optim

In [ ]:
CPU_DEVICE = 'cpu'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
# DEVICE = 'cpu'
DEVICE

In [ ]:
sys.path.append('/media/przemek/ubuntu18/home/przemek/Projects/pp/corn-field-damage/src')
import model_training_v2.common.dataset_preparation as dataset_preparation
import model_training_v2.common.corn_dataset as corn_dataset
import model_training_v2.common.model_definition as model_definition
import model_training_v2.common.model_training_results as model_training_results
import model_training_v2.common.model_training as model_training
import model_training_v2.common.plot_graphs as plot_graphs


from importlib import reload 
reload(dataset_preparation)
reload(corn_dataset)
reload(model_definition)
reload(model_training_results)
reload(model_training)
reload(plot_graphs)


dataset_preparation.init_seeds(778)

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# TILES_BASE_DIR = '/media/data/local/corn/new/tiles_stride_768/'
TILE_SIZE = 512

# BASE_OUTPUT_DIR = '/tmp/aaa/out'

In [ ]:
model_type = model_definition.ModelType.UNET_PLUS_PLUS__EFFICIENT_NET_B3

is_ndvi = False
image_channels = 1 if is_ndvi else 3
model, model_params = model_definition.get_model_with_params(model_type, in_channels=image_channels, tile_size=TILE_SIZE)

print(f'model_params = {model_params}')
# print(model)


MODEL_PATH = '/media/przemek/dane/local/corn/out/from_drive/model_UNET_PLUS_PLUS__EFFICIENT_NET_B3__big_lr_as_always/model.zip'
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()
print(model)

# model.to(DEVICE)

In [ ]:
EXAMPLE_TILE_PATH = '/home/przemek/Desktop/corn/model/example_tile.png'
img_bgr = cv2.imread(EXAMPLE_TILE_PATH)
img = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
# img = img_bgr
plt.imshow(img)

In [ ]:
prepared_img = img.astype('float32')
prepared_img /= 255
prepared_img = prepared_img.transpose(2, 0, 1)  # TODO check
                          
    
input_batch = torch.Tensor(prepared_img).unsqueeze(0)  # aka torch.tensor(img)[np.newaxis, :, :]
print(input_batch.shape)

In [ ]:
with torch.no_grad():
    model_output = model(input_batch)
    
predicted_mask_damage = model_output[0][1].numpy() * 255
plt.imshow(predicted_mask_damage)

In [ ]:
import torch.onnx
import io


In [ ]:
torch.onnx.export(model, inputs, output_dir / 'keypoints_detector.onnx', opset_version=15, input_names=['input'],
                      output_names=['confidence', 'std', 'coords'])


In [ ]:
model.encoder.set_swish(memory_efficient=False)

In [ ]:
ONNX_MPDEL_FILE_PATH = '/home/przemek/Desktop/corn/model/corn_segmentation_model.onnx'

# Export the model
torch.onnx.export(model,               # model being run
                  input_batch,                 # model input (or a tuple for multiple inputs)
                  ONNX_MPDEL_FILE_PATH,   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=16,          # the ONNX version to export the model to
#                   do_constant_folding=True,  # whether to execute constant folding for optimization
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input_rgb_512x512'],   # the model's input names
                  output_names = ['output'], # the model's output names
                 )

In [ ]:
input_batch.shape

In [ ]:
input_batch_numpy = input_batch.numpy()
input_batch_numpy.shape

In [ ]:
import onnxruntime as rt


In [ ]:
sess = rt.InferenceSession(ONNX_MPDEL_FILE_PATH)
input_name = sess.get_inputs()[0].name
input_name
pred_onx = sess.run(None, {input_name: input_batch_numpy})
damaged_area_onnx = pred_onx[0][0][1] * 255

In [ ]:
plt.imshow(damaged_area_onnx)

In [ ]:
ONNX_MPDEL_FILE_PATH

In [ ]:
import onnx

modelonnx = onnx.load(ONNX_MPDEL_FILE_PATH)
for input in modelonnx.graph.input:
    print (input.name, end=": ")
    # get type of input tensor
    tensor_type = input.type.tensor_type
    # check if it has a shape:
    if (tensor_type.HasField("shape")):
        # iterate through dimensions of the shape:
        for d in tensor_type.shape.dim:
            # the dimension may have a definite (integer) value or a symbolic identifier or neither:
            if (d.HasField("dim_value")):
                print (d.dim_value, end=", ")  # known dimension
            elif (d.HasField("dim_param")):
                print (d.dim_param, end=", ")  # unknown dimension with symbolic name
            else:
                print ("?", end=", ")  # unknown dimension with no name
    else:
        print ("unknown rank", end="")
    print()
